In [1]:
import pandas as pd
import networkx as nx
import json
pd.set_option('display.max_columns', None)
from networkx.readwrite import json_graph

In [2]:
df = pd.read_csv(filepath_or_buffer='https://raw.githubusercontent.com/umassdgithub/Fall-2023-DataViz/main/Week-8-ForceSimulator/data/data_scopus.csv')

In [3]:
nodes = df['EID'].values
df=df.fillna(0)
# df.head(2)
# df.shape

In [4]:
graph = nx.Graph()

In [5]:
nodes = {}

for _, row in df.iterrows():
    authors = row['Authors'].split(',')
    author_id = row['Author(s) ID'].split(';')[:-1]
    title = row['Title']
    year = row['Year']
    citations = row['Cited by']
    publisher = row['Publisher']
    authors_wa = row['Authors with affiliations']
    
    for author, aid in zip(authors, author_id):
        if aid:
            nodes = {
                'id': aid,
                'Authors': ';'.join(authors),
                'Title': title,
                'Year': year,
                'Citations': citations,
                'Publisher': publisher,
                'Author with affiliations': authors_wa
            }
            graph.add_node(aid, **nodes)

    

In [8]:
with open("pub_network.json", "w") as f:
    json.dump(json_graph.node_link_data(graph), f)

In [11]:
with open('pub_network.json', 'r') as infile:
    pub_nw = json.load(infile)
    
new_nodes=[]
# Create a NetworkX graph
graph = nx.Graph()

def get_country(x):
    first_aff = x.split(';')[0].strip()
    country = first_aff.split(',')[-1].strip()
    return country


for node_data in pub_nw['nodes']:
    eid = node_data.get('id')
    authors=node_data.get('Authors')
    title = node_data.get('Title')
    year = node_data.get('Year')
    citations = node_data.get('Citations')
    publisher = node_data.get('Publisher')

    authors_wa = str(node_data.get('Author with affiliations'))

    country = get_country(authors_wa)

    graph.add_node(eid,authors=authors, title=title, year=year, citations=citations, publisher=publisher,authors_with_affiliations=authors_wa, country=country)
    
# Generate clusters based on authors' countries
nw_clusters = list(nx.connected_components(graph))

# Assign classes to each cluster
class_mapping = {node: idx for idx, cluster in enumerate(nw_clusters) for node in cluster}
nx.set_node_attributes(graph, class_mapping, 'class')

# Create a dictionary to store co-authorship relationships
for row in df.iterrows():
    authors = row[1]['Author(s) ID'].split(';')
    for i in range(len(authors)):
      for j in range(i+1, len(authors)):
        if(authors[i]!="" and authors[j]!=""):
          graph.add_edge(authors[i], authors[j])

# Save the data as a JSON file with proper character encoding
coauthor_data = {'nodes': [{'id': node, **graph.nodes[node]} for node in graph.nodes()],'links': [{'source': source, 'target': target} for source, target in graph.edges()]}
with open('nw_data.json', 'w', encoding='utf-8') as outfile:
    json.dump(coauthor_data, outfile, ensure_ascii=False)   

Graph with 1552 nodes and 3049 edges
